In [2]:
import numpy as np
import pandas as pd
import sympy as sp
from sympy import Matrix, symbols
from scipy.integrate import nquad
from concurrent.futures import ProcessPoolExecutor
!pip install vegas
import vegas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.1/4.1 MB 31.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 102.2 MB/s eta 0:00:00


#1. Specify Molecule (Define atomic coordinates, nuclear charges and orbitals)

In [1]:
## Molecule (e.g. water)

# N.B. distances are in Bohr radii (this will produce a final ground state energy in Hartrees)
# Atomic coordinates of water taken from my UCL CHEM0028 notes

Molecule = {
    "Atom_label": ["O", "H", "H"],
    "Z": [8, 1, 1],
    "x": [0.00000, 0.00000, 0.00000],
    "y": [0.00000, 1.42757, -1.42757],
    "z": [0.00000, -1.11294, -1.11294]
}

df = pd.DataFrame(Molecule)
print(df)


## Orbitals

# Define symbolic coordinates for integration via Sympy

x, y, z = sp.symbols('x y z')
r = (x, y, z)

def sto_3G_1s(r, coeffs, alphas, center=(0, 0, 0)):
  x, y, z = r
  Rx, Ry, Rz = center
  orbital_func = 0
  for i in range(3):
    orbital_func += coeffs[i]*((2*alphas[i])/sp.pi)**0.75 * sp.exp(-alphas[i] *((x - Rx)**2 + (y - Ry)**2 + (z - Rz)**2))
  return orbital_func

def sto_3G_2s(r, coeffs, alphas, center=(0, 0, 0)):
  x, y, z = r
  Rx, Ry, Rz = center
  orbital_func = 0
  for i in range(3):
    orbital_func += coeffs[i]*((2*alphas[i])/sp.pi)**0.75 * sp.exp(-alphas[i]*((x - Rx)**2 + (y - Ry)**2 + (z - Rz)**2))
  return orbital_func

def sto_3G_2px(r, coeffs, alphas, center=(0, 0, 0)):
  x, y, z = r
  Rx, Ry, Rz = center
  orbital_func = 0
  for i in range(3):
    orbital_func += coeffs[i]*((2**1.5*alphas[i]**2.5)/(sp.pi)**1.5)**0.5*(x - Rx)*sp.exp(-alphas[i]*((x - Rx)**2 + (y - Ry)**2 + (z - Rz)**2))
  return orbital_func

def sto_3G_2py(r, coeffs, alphas, center=(0, 0, 0)):
  x, y, z = r
  Rx, Ry, Rz = center
  orbital_func = 0
  for i in range(3):
    orbital_func += coeffs[i]*((2**1.5*alphas[i]**2.5)/(sp.pi)**1.5)**0.5*(y - Ry)*sp.exp(-alphas[i]*((x - Rx)**2 + (y - Ry)**2 + (z - Rz)**2))
  return orbital_func

def sto_3G_2pz(r, coeffs, alphas, center=(0, 0, 0)):
  x, y, z = r
  Rx, Ry, Rz = center
  orbital_func = 0
  for i in range(3):
    orbital_func += coeffs[i]*((2**1.5*alphas[i]**2.5)/(sp.pi)**1.5)**0.5*((z - Rz))*sp.exp(-alphas[i]*((x - Rx)**2 + (y - Ry)**2 + (z - Rz)**2))
  return orbital_func

n_electrons = sum(Molecule["Z"])
print("number of electrons =", n_electrons)

basis_set = [sto_3G_1s(r, center=(Molecule['x'][0], Molecule['y'][0], Molecule['z'][0]),
        coeffs = [0.15432896, 0.53532814, 0.44463454], alphas = [130.709321, 23.80886605, 6.44360831]),
             sto_3G_2s(r, center=(Molecule['x'][0], Molecule['y'][0], Molecule['z'][0]),
        coeffs = [-0.0999672, 0.39951282, 0.70011546], alphas = [5.0331513, 1.1695961, 0.38038896]),
             sto_3G_2px(r, center=(Molecule['x'][0], Molecule['y'][0], Molecule['z'][0]),
        coeffs = [0.15591627, 0.60768371, 0.39195739], alphas = [5.0331513, 1.1695961, 0.38038896]),
             sto_3G_2py(r, center=(Molecule['x'][0], Molecule['y'][0], Molecule['z'][0]),
        coeffs = [0.15591627, 0.60768371, 0.39195739], alphas = [5.0331513, 1.1695961, 0.38038896]),
             sto_3G_2pz(r, center=(Molecule['x'][0], Molecule['y'][0], Molecule['z'][0]),
        coeffs = [0.15591627, 0.60768371, 0.39195739], alphas = [5.0331513, 1.1695961, 0.38038896]),
             sto_3G_1s(r, center=(Molecule['x'][1], Molecule['y'][1], Molecule['z'][1]),
        coeffs = [0.15432896, 0.53532814, 0.44463454], alphas = [3.4252509, 0.62391372, 0.16885540]),
             sto_3G_1s(r, center=(Molecule['x'][2], Molecule['y'][2], Molecule['z'][2]),
        coeffs = [0.15432896, 0.53532814, 0.44463454], alphas = [3.4252509, 0.62391372, 0.16885540])
]

# N.B. Approximate alpha coefficients taken from https://www.basissetexchange.org/

NameError: name 'pd' is not defined

#2. Calculating T and V using basis states (just as in HFSCF)

In [ ]:
#Constructing T matrix
#precompute laplacians for efficiency

x, y, z, alpha, Rx, Ry, Rz = sp.symbols('x y z alpha Rx Ry Rz')
laplacians = [sp.diff(phi, x, 2) + sp.diff(phi, y, 2) + sp.diff(phi, z, 2) for phi in basis_set]



def t_ij(i, j, basis_set, laplacians):
  integrand = sp.lambdify([[x, y, z]], -0.5 * basis_set[i] * laplacians[j], modules='numpy')
  integrator = vegas.Integrator([[-3, 3], [-3, 3], [-3, 3]])
  t = integrator(integrand, nitn=50, neval=10000).mean
  return t


#Parallelisation needed to compute 7x7 matrix within a few mins

def parallel_T_matrix(basis_set, laplacians):
    n = len(basis_set)
    T = np.zeros((n, n))
    with ProcessPoolExecutor(max_workers=8) as executor:
        futures = [[executor.submit(t_ij, i, j, basis_set, laplacians) for j in range(n)] for i in range(n)]
        for i in range(n):
            for j in range(n):
                T[i, j] = futures[i][j].result()
        # Calculate Oxygen 1s KE separately since numeric integration struggles to capture it
    integrand_sym = -0.5 * basis_set[0] * laplacians[0]  # symbolic expression
    T00_sym = sp.integrate(integrand_sym, (x, -sp.oo, sp.oo), (y, -sp.oo, sp.oo), (z, -sp.oo, sp.oo))
    T[0, 0] = float(T00_sym.evalf())
    return T

T = parallel_T_matrix(basis_set, laplacians)

print(T)

In [ ]:
#Constructing V matrix
x, y, z, alpha, Rx, Ry, Rz = sp.symbols('x y z alpha Rx Ry Rz')


#constructing V_func = -Sigma(Z/abs(r - R)) term:
V_funcs = 0
for i in range(len(Molecule["Z"])):
  V_func = - Molecule["Z"][i] / (sp.sqrt((x - Molecule["x"][i])**2 + (y - Molecule["y"][i])**2 + (z - Molecule["z"][i])**2))
  V_funcs += V_func


#precomuting integrands
integrands = [[sp.lambdify((x, y, z), V_funcs * basis_set[i] * basis_set[j], modules='numpy')
              for j in range(len(basis_set))] for i in range(len(basis_set))]

def v_ij(i, j, basis_set):
  integrand = sp.lambdify([[x, y, z]], V_funcs * basis_set[i] * basis_set[j], modules='numpy')
  integrator = vegas.Integrator([[-3, 3], [-3, 3], [-3, 3]])
  v = integrator(integrand, nitn=50, neval=10000).mean
  return v

def parallel_V_matrix(basis_set):
    n = len(basis_set)
    V = np.zeros((n, n))
    with ProcessPoolExecutor(max_workers=8) as executor:
        futures = [[executor.submit(v_ij, i, j, basis_set) for j in range(n)] for i in range(n)]
        for i in range(n):
            for j in range(n):
                V[i, j] = futures[i][j].result()
    return V

V = parallel_V_matrix(basis_set=basis_set)

print(V)

#3. Define the E_XC functional (e.g. B3LYP) and then determine V_XC via differentiation

#4. Guess the coefficient matrix C using Huckel and then construct the density matrix P. Finally, formulate guess for the density function rho via rho = SUM(P_ij chi_i, chi_j).

#5. Calculating S, U and X such that (U^T S U) = s; (X^T S X) = I.

In [ ]:
# Calculating the overlap integral S
# Write nested function so that it can be reused in the SCF procudure later

def s_ij(i, j, basis_set):
  integrand = sp.lambdify([[x, y, z]], basis_set[i] * basis_set[j], modules='numpy')
  integrator = vegas.Integrator([[-3, 3], [-3, 3], [-3, 3]])
  s = integrator(integrand, nitn=50, neval=10000).mean
  return s

# Compute S elements in parallel
def parallel_S_matrix(basis_set):
    n = len(basis_set)
    S = np.zeros((n, n))
    with ProcessPoolExecutor(max_workers=8) as executor:
        futures = [[executor.submit(s_ij, i, j, basis_set)
                    for j in range(n)] for i in range(n)]
        for i in range(n):
            for j in range(n):
                S[i, j] = futures[i][j].result()
    return S

S = parallel_S_matrix(basis_set)

# Obtaining U and X
eigenvalues, eigenvectors = np.linalg.eig(S)
U = eigenvectors
s_inv_sqrt = np.diag(eigenvalues**-0.5)
X = U @ s_inv_sqrt

S = parallel_S_matrix(basis_set)

sp.pprint(S)

# Check that X^T S X = I
sp.pprint(X.T @ S @ X)

#6. (i) Construct h^KS = T + V + V_e/nuc + V_ee and eigenvalue equation h^KS C = SCe. (ii) Then use transformations from (5.) to solve eigenvectors + eigenenergies of (h^KS)' C' = C' e. (iii) Iterate by transforming C' -> C, reformulating rho, reformulating h^KS and solving for C' again.